In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# The data we are going to use
data="In the town of Athy one Jeremy Lanigan \n Battered away til he hadnt a pound. \nHis father died and made him a man again \n Left him a farm and ten acres of ground. \nHe gave a grand party for friends and relations \nWho didnt forget him when come to the wall, \nAnd if youll but listen Ill make your eyes glisten \nOf the rows and the ructions of Lanigans Ball. \nMyself to be sure got free invitation, \nFor all the nice girls and boys I might ask, \nAnd just in a minute both friends and relations \nWere dancing round merry as bees round a cask. \nJudy ODaly, that nice little milliner, \nShe tipped me a wink for to give her a call, \nAnd I soon arrived with Peggy McGilligan \nJust in time for Lanigans Ball. \nThere were lashings of punch and wine for the ladies, \nPotatoes and cakes; there was bacon and tea, \nThere were the Nolans, Dolans, OGradys \nCourting the girls and dancing away. \nSongs they went round as plenty as water, \nThe harp that once sounded in Taras old hall,\nSweet Nelly Gray and The Rat Catchers Daughter,\nAll singing together at Lanigans Ball. \nThey were doing all kinds of nonsensical polkas \nAll round the room in a whirligig. \nJulia and I, we banished their nonsense \nAnd tipped them the twist of a reel and a jig. \nAch mavrone, how the girls got all mad at me \nDanced til youd think the ceiling would fall. \nFor I spent three weeks at Brooks Academy \nLearning new steps for Lanigans Ball. \nThree long weeks I spent up in Dublin, \nThree long weeks to learn nothing at all,\n Three long weeks I spent up in Dublin, \nLearning new steps for Lanigans Ball. \nShe stepped out and I stepped in again, \nI stepped out and she stepped in again, \nShe stepped out and I stepped in again, \nLearning new steps for Lanigans Ball. \nBoys were all merry and the girls they were hearty \nAnd danced all around in couples and groups, \nTil an accident happened, young Terrance McCarthy \nPut his right leg through miss Finnertys hoops. \nPoor creature fainted and cried Meelia murther, \nCalled for her brothers and gathered them all. \nCarmody swore that hed go no further \nTil he had satisfaction at Lanigans Ball. \nIn the midst of the row miss Kerrigan fainted, \nHer cheeks at the same time as red as a rose. \nSome of the lads declared she was painted, \nShe took a small drop too much, I suppose. \nHer sweetheart, Ned Morgan, so powerful and able, \nWhen he saw his fair colleen stretched out by the wall, \nTore the left leg from under the table \nAnd smashed all the Chaneys at Lanigans Ball. \nBoys, oh boys, twas then there were runctions. \nMyself got a lick from big Phelim McHugh. \nI soon replied to his introduction \nAnd kicked up a terrible hullabaloo. \nOld Casey, the piper, was near being strangled. \nThey squeezed up his pipes, bellows, chanters and all. \nThe girls, in their ribbons, they got all entangled \nAnd that put an end to Lanigans Ball."

In [4]:
# Split the long string per line and put in a list
corpus = data.lower().split("\n")

# Preview the result
print(corpus)

['in the town of athy one jeremy lanigan ', ' battered away til he hadnt a pound. ', 'his father died and made him a man again ', ' left him a farm and ten acres of ground. ', 'he gave a grand party for friends and relations ', 'who didnt forget him when come to the wall, ', 'and if youll but listen ill make your eyes glisten ', 'of the rows and the ructions of lanigans ball. ', 'myself to be sure got free invitation, ', 'for all the nice girls and boys i might ask, ', 'and just in a minute both friends and relations ', 'were dancing round merry as bees round a cask. ', 'judy odaly, that nice little milliner, ', 'she tipped me a wink for to give her a call, ', 'and i soon arrived with peggy mcgilligan ', 'just in time for lanigans ball. ', 'there were lashings of punch and wine for the ladies, ', 'potatoes and cakes; there was bacon and tea, ', 'there were the nolans, dolans, ogradys ', 'courting the girls and dancing away. ', 'songs they went round as plenty as water, ', 'the harp tha

In [46]:
token = Tokenizer()
token.fit_on_texts(corpus)

# grab the total word of the corpus
total_word = len(token.word_index) + 1 # add 1 for the oov token

In [47]:
# make a list of sequences
sequences = []
for sentence in corpus:
  each_word = token.texts_to_sequences([sentence])[0]
  # each_word = sentence.split()
  for i in range(1, len(each_word)):
    sequences.append(each_word[:i+1])
# print(sequences)

# Get the Maximum length sentence in the list of sentences
max_length = max([len(sequence) for sequence in sequences])
# print(max_length)

# Pad all the sequences
padded_sequences = np.array(pad_sequences(sequences, maxlen = max_length, truncating = 'pre', padding = 'pre'))
# print(padded_sequences)

# Create inputs and label by splitting the last token in the subphrases
xs, labels = padded_sequences[:,:-1],padded_sequences[:,-1]
# Convert the label into one-hot arrays
ys = tf.keras.utils.to_categorical(labels, num_classes=total_word)

In [48]:
# Build the model
model = Sequential([
          Embedding(total_word, 64, input_length=max_length-1),
          Bidirectional(LSTM(20)),
          Dense(total_word, activation='softmax')
])

# Use categorical crossentropy because this is a multi-class problem
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 10, 64)            16832     
                                                                 
 bidirectional_2 (Bidirectio  (None, 40)               13600     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 263)               10783     
                                                                 
Total params: 41,215
Trainable params: 41,215
Non-trainable params: 0
_________________________________________________________________


In [49]:
# Train the model
history = model.fit(xs, ys, epochs=500)

Epoch 1/500
15/15 [==============================] - 6s 163ms/step - loss: 5.5693 - accuracy: 0.0331
Epoch 2/500
15/15 [==============================] - 3s 184ms/step - loss: 5.5467 - accuracy: 0.0508
Epoch 3/500
15/15 [==============================] - 1s 72ms/step - loss: 5.4928 - accuracy: 0.0508
Epoch 4/500
15/15 [==============================] - 0s 20ms/step - loss: 5.3394 - accuracy: 0.0508
Epoch 5/500
15/15 [==============================] - 0s 20ms/step - loss: 5.1570 - accuracy: 0.0508
Epoch 6/500
15/15 [==============================] - 1s 59ms/step - loss: 5.0863 - accuracy: 0.0508
Epoch 7/500
15/15 [==============================] - 1s 46ms/step - loss: 5.0465 - accuracy: 0.0508
Epoch 8/500
15/15 [==============================] - 1s 47ms/step - loss: 5.0150 - accuracy: 0.0508
Epoch 9/500
15/15 [==============================] - 0s 32ms/step - loss: 4.9865 - accuracy: 0.0618
Epoch 10/500
15/15 [==============================] - 0s 20ms/step - loss: 4.9554 - accuracy: 0.05

In [50]:
# Define seed text
seed_text = "Laurence went to Dublin"

# Define total words to predict
next_words = 100

# Loop until desired length is reached
for _ in range(next_words):

  # Convert the seed text to a token sequence
  token_list = token.texts_to_sequences([seed_text])[0]

  # Pad the sequence
  token_list = pad_sequences([token_list], maxlen=max_length-1, padding='pre')

  # Feed to the model and get the probabilities for each index
  probabilities = model.predict(token_list)

  # Pick a random number from [1,2,3]
  choice = np.random.choice([1,2,3])

  # Sort the probabilities in ascending order
  # and get the random choice from the end of the array
  predicted = np.argsort(probabilities)[0][-choice]

  # Ignore if index is 0 because that is just the padding.
  if predicted != 0:

    # Look up the word associated with the index.
    output_word = token.index_word[predicted]

    # Combine text with output
    seed_text += " " + output_word

# Print the result
print(seed_text)

1/1 [==============================] - 0s 26ms/step
Laurence went to Dublin around for her brothers and able relations be mccarthy ladies ladies reel a and jig mchugh terrible eyes rose glisten hall give had rose rose entangled rose away entangled a jig ask happened murther rose glisten glisten rose nonsensical mccarthy call happened mccarthy hall satisfaction him him a mchugh strangled casey mavrone cask a call hullabaloo happened eyes rose hearty nonsensical hall lanigan water hadnt lanigans bees steps time a lanigans round rose odaly odaly round steps again round again round a give to her me lanigans ball ball rose rose away entangled ball entangled away rose rose rose a
